In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from overcomplete.sae import JumpSAE, train_sae
import pandas as pd

In [2]:
train_path = "./logs/activation.pt"
ckpt = torch.load(train_path)

fc1 = ckpt["fc1"]
fc1_activations = ckpt["fc1_activations"]
labels = ckpt["labels"]

batch_size = 1024
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

dataloader = torch.utils.data.DataLoader(TensorDataset(fc1), batch_size=batch_size, shuffle=True)

/tmp/ipykernel_2293530/3713506127.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(train_path)


In [4]:
sae = JumpSAE(fc1.shape[-1], nb_concepts=256,
              bandwith=1e-2, kernel='silverman', device='cuda')

optimizer = torch.optim.Adam(sae.parameters(), lr=3e-3)

desired_sparsity = 0.05

def criterion(x, x_hat, pre_codes, codes, dictionary):
  # here we directly use the thresholds of the model to control the sparsity
  loss = (x - x_hat).square().mean()

  sparsity = (codes > 0).float().mean().detach()
  if sparsity > desired_sparsity:
    # if we are not sparse enough, increase the thresholds levels
    loss -= sae.thresholds.sum()

  return loss

logs = train_sae(sae, dataloader, criterion, optimizer, nb_epochs=30, device='cuda')

Epoch[1/30], Loss: 747.2267, R2: 0.8285, L0: 92.1096, Dead Features: 7.8%, Time: 208.8014 seconds
Epoch[2/30], Loss: 697.1829, R2: 0.9881, L0: 100.2846, Dead Features: 53.9%, Time: 166.4026 seconds
Epoch[3/30], Loss: 650.5390, R2: 0.9936, L0: 103.5450, Dead Features: 53.9%, Time: 188.6200 seconds
Epoch[4/30], Loss: 604.4399, R2: 0.9950, L0: 103.5547, Dead Features: 53.9%, Time: 189.4940 seconds
Epoch[5/30], Loss: 558.5914, R2: 0.9957, L0: 103.4471, Dead Features: 53.9%, Time: 195.2696 seconds
Epoch[6/30], Loss: 512.8776, R2: 0.9962, L0: 103.2384, Dead Features: 53.9%, Time: 172.2337 seconds
Epoch[7/30], Loss: 467.2673, R2: 0.9965, L0: 102.8322, Dead Features: 53.9%, Time: 185.0890 seconds
Epoch[8/30], Loss: 421.7168, R2: 0.9969, L0: 102.4130, Dead Features: 54.3%, Time: 211.0061 seconds
Epoch[9/30], Loss: 376.1782, R2: 0.9971, L0: 101.7326, Dead Features: 54.3%, Time: 172.6667 seconds
Epoch[10/30], Loss: 330.6875, R2: 0.9974, L0: 100.4855, Dead Features: 54.7%, Time: 167.2528 seconds
E